In [ ]:
import json
import pandas as pd
import requests
from lattice import Connection


mode = 'prod'
connection = Connection(mode)
serv = connection.server

In [ ]:
def query_donors(group):
    if group.startswith('LATDS'):
        query = '&dataset=/datasets/' + group + '/'
    else:
        query = '&lab.name=' + group

    obj_dict = {'human-postnatal-donors': 'HumanPostnatalDonor', 'human-prenatal-donors': 'HumanPrenatalDonor'}
    donors = {'human-postnatal-donors': [], 'human-prenatal-donors': []}
    donors_temp = []
    blank = []


    url = serv + f'report/?type=Library{query}&field=donors.@id&limit=all'
    r = requests.get(url, auth=connection.auth).json()
    for l in r['@graph']:
        donors_temp.extend(d['@id'] for d in l['donors'])
    donors_temp = set(donors_temp)
    for d in donors_temp:
        objtype = d.split('/')[1]
        acc = d.split('/')[2]
        donors[objtype].append(acc)
    for k,v in donors.items():
        if v:
            url = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v)
            if len(url) > 8000: #max url length is 8205
                mid = len(v)//2
                url1 = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v[:mid])
                blank.append((str(len(v[:mid])) + ' donors', url1))
                #print(str(len(v[:mid])) + ' donors')
                #print(url1)
                url2 = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v[mid:])
                blank.append((str(len(v[mid:])) + ' donors', url2))
                #print(str(len(v[mid:])) + ' donors')
                #print(url2)
            else:
                blank.append((str(len(v)) + ' donors', url))
                #print(str(len(v)) + ' donors')
                #print(url)
    if blank:
        count = 0
        print(str(len(blank)) + ' URL(s)')
        for i in blank:
            count += 1
            print(f'{str(count)}: {i[0]}')
            print(i[1])

In [ ]:
#query Datasets by Accession
query_donors('LATDS977NKW')

In [ ]:
#query Labs by Name
query_donors('jason-spence')